In [ ]:
# Installs

!pip install paddlepaddle -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install "paddleocr>=2.0.1"
!pip install pybind11
!pip install fastwer
!pip install transformers --quiet
!pip install Levenshtein --quiet

DEPRECATION: Python 2.7 reached the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 is no longer maintained. pip 21.0 will drop support for Python 2.7 in January 2021. More details about Python 2 support in pip can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support pip 21.0 will remove support for this functionality.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     |████████████████████████████████| 168.5 MB 32 kB/s s eta 0:00:01
     |████████████████████████████████| 2.1 MB 8.9 MB/s eta 0:00:01
     |████████████████████████████████| 63 kB 1.4 MB/s eta 0:00:011
     |████████████████████████████████| 17.0 MB 487 kB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 5.0 MB/s eta 0:00:01
     |████████████████████████████████| 149 kB 8.4 MB/s eta 0:00:01
     |████████████████████████████████| 140 kB 4.4 MB/s eta 0:00:01
     |████████████████████████████████| 178 kB 228 kB/s eta 0:00:01
    

In [3]:
# Imports

import os
import shutil
from PIL import Image

import torch
from io import BytesIO
from transformers import VisionEncoderDecoderModel, VisionEncoderDecoderConfig, DonutProcessor
import json
from tqdm.notebook import tqdm

In [4]:
!git clone https://github.com/Losyash/SROIE-datasetv2.git

Cloning into 'SROIE-datasetv2'...
remote: Enumerating objects: 3876, done.
remote: Counting objects: 100% (975/975), done.
remote: Compressing objects: 100% (971/971), done.
remote: Total 3876 (delta 3), reused 971 (delta 2), pack-reused 2901
Receiving objects: 100% (3876/3876), 429.85 MiB | 21.08 MiB/s, done.
Resolving deltas: 100% (93/93), done.
Updating files: 100% (3894/3894), done.


In [6]:
sroie_root_dir = './SROIE-datasetv2/'

In [7]:
# Make dir for test predicted
os.mkdir(os.path.join(sroie_root_dir, 'test_predicted'))
os.mkdir(os.path.join(sroie_root_dir, 'test_predicted/box/'))
os.mkdir(os.path.join(sroie_root_dir, 'test_predicted/entities/'))

In [8]:
sroie_test_img_dir = os.path.join(sroie_root_dir, 'test/img')
sroie_test_pred_ent_dir = os.path.join(sroie_root_dir, 'test_predicted/entities')
sroie_test_pred_box_dir = os.path.join(sroie_root_dir, 'test_predicted/box')

In [9]:
# Boxes

from paddleocr import PaddleOCR
ocr = PaddleOCR(use_angle_cls=True, lang='en')

download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_det_infer.tar to /root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer/en_PP-OCRv3_det_infer.tar


100%|██████████| 4.00M/4.00M [00:05<00:00, 794kiB/s] 


download https://paddleocr.bj.bcebos.com/PP-OCRv3/english/en_PP-OCRv3_rec_infer.tar to /root/.paddleocr/whl/rec/en/en_PP-OCRv3_rec_infer/en_PP-OCRv3_rec_infer.tar


100%|██████████| 9.96M/9.96M [00:10<00:00, 987kiB/s] 


download https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar to /root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer/ch_ppocr_mobile_v2.0_cls_infer.tar


100%|██████████| 2.19M/2.19M [00:06<00:00, 317kiB/s]

[2023/01/20 14:15:01] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thresh=0.5, e2e_pgnet_valid_set

In [10]:
# Ents

processor = DonutProcessor.from_pretrained("unstructuredio/donut-base-sroie")
pretrained_model = VisionEncoderDecoderModel.from_pretrained("unstructuredio/donut-base-sroie")

device = "cuda" if torch.cuda.is_available() else "cpu"
pretrained_model.to(device)

Downloading:   0%|          | 0.00/361 [00:00<?, ?B/s]

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.


Downloading:   0%|          | 0.00/497 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.01M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/257 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/485 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/809M [00:00<?, ?B/s]

VisionEncoderDecoderModel(
  (encoder): DonutSwinModel(
    (embeddings): DonutSwinEmbeddings(
      (patch_embeddings): DonutSwinPatchEmbeddings(
        (projection): Conv2d(3, 128, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DonutSwinEncoder(
      (layers): ModuleList(
        (0): DonutSwinStage(
          (blocks): ModuleList(
            (0): DonutSwinLayer(
              (layernorm_before): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
              (attention): DonutSwinAttention(
                (self): DonutSwinSelfAttention(
                  (query): Linear(in_features=128, out_features=128, bias=True)
                  (key): Linear(in_features=128, out_features=128, bias=True)
                  (value): Linear(in_features=128, out_features=128, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )

In [11]:
# Fx

def make_box_file(file_name):

    result = ocr.ocr(f'{sroie_test_img_dir}/{file_name}', cls=True)
    file_name_2 = file_name.split('.')[0] + '-1.txt'

    with open(f'{sroie_test_pred_box_dir}/{file_name_2}', "w") as file:
        for i in range(len(result)):
            res = result[i]

            for line in res:
                data = []

                for j in range(len(line[0])):
                    coords = line[0][j]

                    data.append(str(int(coords[0])))
                    data.append(str(int(coords[1])))

                data.append(str(line[1][0]).upper())
                file.write((',').join(data) + '\n')    

def make_entities_file(file_name):
    global pretrained_model, processor

    path_to_test_img = os.path.join(sroie_test_img_dir, file_name)
    image = Image.open(path_to_test_img).convert("RGB")

    task_prompt = f"<s>"
    pixel_values = processor(image, return_tensors="pt").pixel_values
    decoder_input_ids = processor.tokenizer(task_prompt, add_special_tokens=False, return_tensors="pt").input_ids

    # run inference
    outputs = pretrained_model.generate(
        pixel_values.to(device),
        decoder_input_ids=decoder_input_ids.to(device),
        max_length=pretrained_model.decoder.config.max_position_embeddings,
        early_stopping=True,
        pad_token_id=processor.tokenizer.pad_token_id,
        eos_token_id=processor.tokenizer.eos_token_id,
        use_cache=True,
        num_beams=1,
        bad_words_ids=[[processor.tokenizer.unk_token_id]],
        return_dict_in_generate=True,
    )

    # process output
    prediction = processor.batch_decode(outputs.sequences)[0]
    prediction = processor.token2json(prediction)
    prediction = dict(sorted(prediction.items()))

    for key in ['address', 'company', 'date', 'total']:
        if key not in prediction:
            prediction[key] = ''

    file_name_2 = file_name.split('.')[0] + '-2.txt'

    path_to_test_pred_file = os.path.join(sroie_test_pred_ent_dir, file_name_2)
    with open(path_to_test_pred_file, "w") as fp:
        json.dump(prediction, fp) 

In [ ]:
# Very long time... stoppped for now (making GPU version)

for file_name in tqdm(os.listdir(sroie_test_img_dir)):
    make_box_file(file_name)
    make_entities_file(file_name)

print('Done')

In [13]:
# Copy result to cloud

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/SROIE-datasetv2/test_predicted /content/drive/MyDrive/Hackaton2023